# Performance evaluation (with plots)

This code allows you to run different algorithms on a certain set of datasets, obtain their AUC-ROC and AUC-PR scores and obtain the associated plot highlighting the anomaly scores, as well as the original anomalous time series.

## Install necessary libraries

In [59]:
!pip install docker
!pip install timeeval --no-deps
!pip install durations numpyencoder distributed prts

    scikit-learn (>=0.24.*)
                  ~~~~~~~^


## Import libraries

In [60]:
import os
import json
import subprocess
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
from timeeval.utils.window import ReverseWindowing
import signal
import threading
import time
from functools import wraps
import platform
import psutil
import gc
import traceback
from typing import Optional
from concurrent.futures import TimeoutError

## Check if Docker is installed and running (if not install it from here: https://docs.docker.com/engine/install/ and open the app to allow Docker to run in the background)

In [61]:
def verify_docker():
    if not shutil.which("docker"):
        raise EnvironmentError("Docker is not installed or not in your PATH. Please install Docker.")

## Build Docker images

In [62]:
def pull_docker_images():
    images = [
        "ghcr.io/timeeval/cblof:0.3.0",
        "ghcr.io/timeeval/cof:0.3.0",
        "ghcr.io/timeeval/kmeans:0.3.0",
        "ghcr.io/timeeval/knn:0.3.0",
        "ghcr.io/timeeval/lof:0.3.0",
        "ghcr.io/timeeval/phasespace_svm:0.3.0",
        "ghcr.io/timeeval/subsequence_lof:0.3.0",
        "ghcr.io/timeeval/valmod:0.3.0"
    ]

    # Build Docker images
    for image in images:
        try:
            subprocess.run(["docker", "pull", "--platform", "linux/amd64", image], check=True)
            print(f"Successfully pulled {image}.")
        except subprocess.CalledProcessError as e:
            print(f"Failed to pull {image}: {e}")


pull_docker_images()

0.3.0: Pulling from timeeval/cblof
Digest: sha256:4397c29daaac6a99d8cfc572fec055bf81a6bcad43cadc503c85443cd422f9be
Status: Image is up to date for ghcr.io/timeeval/cblof:0.3.0
ghcr.io/timeeval/cblof:0.3.0
Successfully pulled ghcr.io/timeeval/cblof:0.3.0.
0.3.0: Pulling from timeeval/cof
Digest: sha256:b6d9200d41641cf443bed91f3498b3b26f36b84d8fcb533f561c1279a7806352
Status: Image is up to date for ghcr.io/timeeval/cof:0.3.0
ghcr.io/timeeval/cof:0.3.0
Successfully pulled ghcr.io/timeeval/cof:0.3.0.
0.3.0: Pulling from timeeval/kmeans
Digest: sha256:acd5f5f6c8a644a02fa755e0fc274d323263f2f34e43e496fc174224b03e0ec8
Status: Image is up to date for ghcr.io/timeeval/kmeans:0.3.0
ghcr.io/timeeval/kmeans:0.3.0
Successfully pulled ghcr.io/timeeval/kmeans:0.3.0.
0.3.0: Pulling from timeeval/knn
Digest: sha256:a9bb4f68b291c0f91654c1275da9089e91ddff2203ecfbbc504af12504f461be
Status: Image is up to date for ghcr.io/timeeval/knn:0.3.0
ghcr.io/timeeval/knn:0.3.0
Successfully pulled ghcr.io/timeeval/knn

## Define paths

In [63]:
# Define paths to the algorithms
ALGORITHMS = {
    "CBLOF": "ghcr.io/timeeval/cblof:0.3.0",
    "COF": "ghcr.io/timeeval/cof:0.3.0",
    "kMeans": "ghcr.io/timeeval/kmeans:0.3.0",
    "KNN": "ghcr.io/timeeval/knn:0.3.0",
    "LOF": "ghcr.io/timeeval/lof:0.3.0",
    "PS-SVM": "ghcr.io/timeeval/phasespace_svm:0.3.0",
    "Sub-LOF": "ghcr.io/timeeval/subsequence_lof:0.3.0",
    "VALMOD": "ghcr.io/timeeval/valmod:0.3.0"
}

## General function allowing you to run the algorithms

In [64]:
def run_algorithm(algorithm_name, data_path, execution_type="execute", custom_parameters=None):

    # Check if Docker image of algorithm was specified
    if algorithm_name not in ALGORITHMS:
        raise ValueError(f"Algorithm '{algorithm_name}' not found in ALGORITHMS dictionary.")
    
    # Check if the data file exists
    config = {
        "executionType": execution_type,
        "dataInput": "/" + data_path,
        "dataOutput": "/results/anomaly_scores.csv",
        "modelInput": "/results/model.pkl" if execution_type == "train" else "/results/trained_model.pkl",
        "modelOutput": "/results/trained_model.pkl" if execution_type == "train" else "/results/model.pkl",
        "customParameters": custom_parameters or {}
    }
    
    config_json = json.dumps(config)
    
    # Build Docker command
    cmd = [
        "docker", "run", "--rm",
        "-v", f"{os.getcwd()}/data:/data:ro",
        "-v", f"{os.getcwd()}/results:/results:rw",
        "--platform", "linux/amd64",
        ALGORITHMS[algorithm_name], "execute-algorithm", config_json
    ]
    
    # Execute Docker command
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    # Check for errors
    if result.returncode != 0:
        print(f"Error running {algorithm_name}: {result.stderr}")
        raise RuntimeError(f"Execution failed for {algorithm_name}")
    
    # Load and return anomaly scores
    scores_path = "./results/anomaly_scores.csv"
    scores = np.loadtxt(scores_path, delimiter=",")
    return scores

## This class + function handle timeout errors, we have introduced this due to some running for over 60s for a single dataset

In [65]:
class TimeoutError(Exception):
    pass

def timeout_handler(timeout):
    """Decorator that implements timeout for functions on both Unix and Windows"""
    def decorator(func):
        if platform.system() != 'Windows':
            # Unix-based systems can use signal
            def handler(signum, frame):
                raise TimeoutError(f"Function '{func.__name__}' timed out after {timeout} seconds")

            @wraps(func)
            def wrapper(*args, **kwargs):
                # Set signal handler
                old_handler = signal.signal(signal.SIGALRM, handler)
                signal.alarm(timeout)
                try:
                    result = func(*args, **kwargs)
                finally:
                    # Restore old handler and disable alarm
                    signal.alarm(0)
                    signal.signal(signal.SIGALRM, old_handler)
                return result
            
        else:
            # Windows implementation using threading
            @wraps(func)
            def wrapper(*args, **kwargs):
                result = []
                error = []
                
                def worker():
                    try:
                        result.append(func(*args, **kwargs))
                    except Exception as e:
                        error.append(e)
                
                thread = threading.Thread(target=worker)
                thread.daemon = True
                thread.start()
                thread.join(timeout)
                
                if thread.is_alive():
                    raise TimeoutError(f"Function '{func.__name__}' timed out after {timeout} seconds")
                if error:
                    raise error[0]
                return result[0]
                
        return wrapper
    return decorator

## Same idea as previous block, only for memory related issues

In [66]:
class MemoryError(Exception):
    pass

def check_memory_usage(threshold_percent: float = 95.0) -> None:
    """Check if memory usage is above threshold"""
    memory_percent = psutil.Process(os.getpid()).memory_percent()
    if memory_percent > threshold_percent:
        raise MemoryError(f"Memory usage too high: {memory_percent:.1f}%")

def safe_load_data(file_path: str) -> Optional[pd.DataFrame]:
    """Safely load data with error handling"""
    try:
        return pd.read_csv(file_path)
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")
        return None

## Function that evaluates the algorithm based on the produced anomaly scores

In [67]:
def evaluate_algorithm(scores, data_path):
    # Load test data
    data_test = pd.read_csv(data_path)
    
    # Ensure 'is_anomaly' column exists in the data
    if 'is_anomaly' not in data_test.columns:
        raise ValueError(f"The test data at {data_path} must contain an 'is_anomaly' column.")

    # Extract the anomaly labels
    anomalies = data_test['is_anomaly']

    # Calculate AUC-ROC and AUC-PR
    auc_roc = roc_auc_score(anomalies, scores)
    precision, recall, _ = precision_recall_curve(anomalies, scores)
    auc_pr = auc(recall, precision)
    return auc_roc, auc_pr

## Some algorithms require post-processing. To find out the exact details of which ones need this, head to https://github.com/TimeEval/TimeEval-algorithms

In [68]:
# Post-processing for Sub-LOF
def post_sLOF(scores: np.ndarray, args: dict) -> np.ndarray:
    window_size = args.get("hyper_params", {}).get("window_size", 100)
    return ReverseWindowing(window_size=window_size).fit_transform(scores)

# Post-processing for VALMOD
def post_valmod(scores: np.ndarray, args: dict) -> np.ndarray:
    window_min = args.get("hyper_params", {}).get("min_anomaly_window_size", 30)
    window_min = max(window_min, 4)
    return ReverseWindowing(window_size=window_min).fit_transform(scores)

# Run for entire GutenTAG dataset

In [69]:
# Define paths
base_path = "data/GutenTag" # Change this to the base path of your dataset
results_summary = []

# Colors for methods (consistent across datasets)
METHOD_COLORS = {
    "CBLOF": "blue",
    "COF": "orange",
    "kMeans": "red",
    "KNN": "purple",
    "LOF": "brown",
    "PS-SVM": "pink",
    "Sub-LOF": "yellow",
    "VALMOD": "green",
}

# Create directory for plots
try:
    os.makedirs("./results/plots", exist_ok=True)
except Exception as e:
    print(f"Error creating directories: {e}")
    exit(1)

# Recursively find all test.csv files
test_files = []
try:
    for root, _, files in os.walk(base_path):
        for file in files:
            if file.endswith("test.csv"):
                test_files.append(os.path.join(root, file))
except Exception as e:
    print(f"Error finding test files: {e}")
    exit(1)

if not test_files:
    print("No test files found!")
    exit(1)

# Process each test.csv file
for test_file in test_files:
    folder = os.path.basename(os.path.dirname(test_file))
    print(f"\nProcessing file: {test_file} in folder: {folder}")
    
    # Load test dataset with error handling
    try:
        data_test = safe_load_data(test_file)
        if data_test is None:
            print(f"Skipping file {test_file} due to loading error")
            continue
        
        x_values = range(len(data_test))
    except Exception as e:
        print(f"Error preparing data for {test_file}: {e}")
        continue
    
    # Initialize results for the file
    file_results = []
    anomaly_scores_dict = {}
    
    for algo_name in ALGORITHMS.keys():
        print(f"\nRunning {algo_name} for {folder}...")
        
        # Clear memory before each algorithm
        gc.collect()
        
        try:
            # Check memory usage before starting
            check_memory_usage()
            
            # Set algorithm parameters
            try:
                custom_params = {
                    "CBLOF": {"n_clusters": 50, "random_state": 42},
                    "COF": {"n_neighbors": 50, "random_state": 42},
                    "kMeans": {"n_clusters": 50, "anomaly_window_size": 100, "random_state": 42},
                    "KNN": {"n_neighbors": 50, "leaf_size": 20, "random_state": 42},
                    "LOF": {"n_neighbors": 50, "leaf_size": 20, "random_state": 42},
                    "PS-SVM": {"random_state": 42},
                    "STAMP": {"window_size": 100, "verbose": 0, "random_state": 42},
                    "STOMP": {"window_size": 100, "verbose": 0, "random_state": 42},
                    "Sub-LOF": {"n_neighbors": 50, "leaf_size": 20, "random_state": 42},
                    "VALMOD": {"verbose": 0, "random_state": 42}
                }.get(algo_name, {})
            except Exception as e:
                print(f"Error setting parameters for {algo_name}: {e}")
                continue

            if platform.system() != 'Windows':
                # Unix implementation using signal
                def timeout_handler(signum, frame):
                    raise TimeoutError(f"Operation timed out after 60 seconds for {algo_name}")

                # Set up signal handler
                signal.signal(signal.SIGALRM, timeout_handler)
                signal.alarm(60)

                try:
                    # Run algorithm
                    scores = run_algorithm(algo_name, test_file, custom_parameters=custom_params)
                    check_memory_usage()  # Check memory after algorithm run
                    
                    # Post-processing (if needed)
                    if algo_name in "STAMP":
                        scores = post_stamp(scores, custom_params)
                    elif algo_name == "STOMP":
                        scores = post_stomp(scores, custom_params)
                    elif algo_name == "Sub-LOF":
                        scores = post_sLOF(scores, custom_params)
                    elif algo_name == "VALMOD":
                        scores = post_valmod(scores, custom_params)
                    
                    check_memory_usage()  # Check memory after post-processing
                
                finally:
                    signal.alarm(0)  # Disable the alarm

            else:
                # Windows implementation using threading
                result = []
                error = []

                def worker():
                    try:
                        # Run algorithm
                        scores = run_algorithm(algo_name, test_file, custom_parameters=custom_params)
                        check_memory_usage()
                        
                        # Post-processing (if needed)
                        if algo_name == "STAMP":
                            scores = post_stamp(scores, custom_params)
                        elif algo_name == "STOMP":
                            scores = post_stomp(scores, custom_params)
                        elif algo_name == "Sub-LOF":
                            scores = post_sLOF(scores, custom_params)
                        elif algo_name == "VALMOD":
                            scores = post_valmod(scores, custom_params)
                        
                        check_memory_usage()
                        result.append(scores)
                    except Exception as e:
                        error.append(e)
                        traceback.print_exc()  # Print full traceback

                thread = threading.Thread(target=worker)
                thread.daemon = True
                thread.start()
                thread.join(60)  # 60 second timeout

                if thread.is_alive():
                    raise TimeoutError(f"Operation timed out after 60 seconds for {algo_name}")
                if error:
                    raise error[0]
                scores = result[0]

            # Evaluate algorithm
            try:
                auc_roc, auc_pr = evaluate_algorithm(scores, test_file)
                print(f"{algo_name} - AUC-ROC: {auc_roc:.3f}, AUC-PR: {auc_pr:.3f}")
                
                # Store results
                file_results.append({"Algorithm": algo_name, "AUC-ROC": auc_roc, "AUC-PR": auc_pr})
                
                # Normalize scores for plotting
                min_val, max_val = np.min(scores), np.max(scores)
                scores = np.zeros_like(scores) if min_val == max_val else (scores - min_val) / (max_val - min_val)
                anomaly_scores_dict[algo_name] = scores
            
            except Exception as e:
                print(f"Error in evaluation phase for {algo_name}: {e}")
                traceback.print_exc()
                continue
        
        except MemoryError as me:
            print(f"Memory error running {algo_name} for {folder}: {me}")
            gc.collect()  # Force garbage collection
            continue
        except TimeoutError as te:
            print(f"Timeout error: {te}")
            continue
        except Exception as e:
            print(f"Unexpected error running {algo_name} for {folder}: {e}")
            traceback.print_exc()
            continue
    
    # Plot results for this file
    try:
        if not anomaly_scores_dict:  # Skip plotting if no algorithms succeeded
            print(f"Skipping plots for {folder} as no algorithms completed successfully")
            continue
            
        plt.figure(figsize=(12, 8))
        
        # Subplot 1: Test time series
        plt.subplot(2, 1, 1)
        for col in data_test.iloc[:, 1:-1].columns:
            plt.plot(x_values, data_test[col], label=f"{col}")
        
        anomaly_mask = data_test.iloc[:, -1] == 1
        anomaly_regions = data_test[anomaly_mask].index
        if not anomaly_regions.empty:
            start = None
            for i in range(len(anomaly_regions)):
                if start is None:
                    start = anomaly_regions[i]
                if i == len(anomaly_regions) - 1 or anomaly_regions[i + 1] != anomaly_regions[i] + 1:
                    end = anomaly_regions[i]
                    plt.axvspan(start, end, color="red", alpha=0.3, 
                              label="Anomaly" if start == anomaly_regions[0] else "")
                    start = None

        plt.title(f"Test Time Series - {folder}")
        plt.xlabel("Time")
        plt.ylabel("Value")
        plt.legend()
        
        # Subplot 2: Anomaly scores
        plt.subplot(2, 1, 2)
        for algo_name, scores in anomaly_scores_dict.items():
            plt.plot(x_values, scores, label=f"{algo_name}", 
                    color=METHOD_COLORS.get(algo_name, "gray"))
        plt.title("Anomaly Scores")
        plt.xlabel("Time")
        plt.ylabel("Score")
        plt.legend()
        
        # Save plot
        try:
            plot_filename = os.path.join("./results/plots", 
                                       f"{folder}_{os.path.basename(test_file)}_scores.png")
            plt.tight_layout()
            plt.savefig(plot_filename)
        except Exception as e:
            print(f"Error saving plot for {folder}: {e}")
        finally:
            plt.close()
            
    except Exception as e:
        print(f"Error creating plots for {folder}: {e}")
        traceback.print_exc()
        continue
    
    # Append results for this file
    if file_results:  # Only append if we have results
        results_summary.append({"File": test_file, "Results": file_results})

# Compute and save average AUC scores
try:
    if results_summary:  # Only process if we have results
        average_auc_scores = {}
        for result in results_summary:
            for algo_result in result["Results"]:
                algo_name = algo_result["Algorithm"]
                auc_score = algo_result["AUC-ROC"]
                if algo_name not in average_auc_scores:
                    average_auc_scores[algo_name] = []
                average_auc_scores[algo_name].append(auc_score)

        # Compute averages
        average_auc_scores = {algo: np.mean(scores) 
                            for algo, scores in average_auc_scores.items()}

        # Save summary results
        try:
            results_df = pd.DataFrame(results_summary)
            results_df.to_csv("./results/summary_results.csv", index=False)
        except Exception as e:
            print(f"Error saving summary results: {e}")

        # Plot average AUC scores
        try:
            plt.figure(figsize=(10, 6))
            plt.bar(average_auc_scores.keys(), average_auc_scores.values(),
                   color=[METHOD_COLORS.get(algo, "gray") 
                         for algo in average_auc_scores.keys()])
            plt.title("Average AUC-ROC Scores by Algorithm")
            plt.xlabel("Algorithm")
            plt.ylabel("Average AUC-ROC")
            plt.xticks(rotation=45, ha="right")
            plt.tight_layout()
            plt.savefig("./results/plots/average_auc_scores.png")
            plt.close()
        except Exception as e:
            print(f"Error creating average AUC score plot: {e}")
    else:
        print("No results were generated to summarize!")

except Exception as e:
    print(f"Error in final summary and plotting: {e}")
    traceback.print_exc()

# Notify completion
print("\nExecution complete. Results and plots saved to './results'.")


Processing file: data/generated_data/dataset_7.test.csv in folder: generated_data

Running CBLOF for generated_data...
CBLOF - AUC-ROC: 0.535, AUC-PR: 0.026

Running COF for generated_data...
COF - AUC-ROC: 0.401, AUC-PR: 0.020

Running kMeans for generated_data...
kMeans - AUC-ROC: 0.588, AUC-PR: 0.033

Running KNN for generated_data...
KNN - AUC-ROC: 0.460, AUC-PR: 0.025

Running LOF for generated_data...
LOF - AUC-ROC: 0.420, AUC-PR: 0.020

Running PS-SVM for generated_data...
PS-SVM - AUC-ROC: 0.561, AUC-PR: 0.033

Running STAMP for generated_data...
Error in evaluation phase for STAMP: Input contains infinity or a value too large for dtype('float64').

Running STOMP for generated_data...


Traceback (most recent call last):
  File "/var/folders/xh/bm_l2jz916s7t2p3pwrcr9y80000gn/T/ipykernel_25469/281630419.py", line 159, in <module>
    auc_roc, auc_pr = evaluate_algorithm(scores, test_file)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/xh/bm_l2jz916s7t2p3pwrcr9y80000gn/T/ipykernel_25469/2679600840.py", line 13, in evaluate_algorithm
    auc_roc = roc_auc_score(anomalies, scores)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_ranking.py", line 619, in roc_auc_score
    y_score = check_array(y_score, ensure_2d=False)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/l

Error in evaluation phase for STOMP: Input contains infinity or a value too large for dtype('float64').

Running Sub-LOF for generated_data...


Traceback (most recent call last):
  File "/var/folders/xh/bm_l2jz916s7t2p3pwrcr9y80000gn/T/ipykernel_25469/281630419.py", line 159, in <module>
    auc_roc, auc_pr = evaluate_algorithm(scores, test_file)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/xh/bm_l2jz916s7t2p3pwrcr9y80000gn/T/ipykernel_25469/2679600840.py", line 13, in evaluate_algorithm
    auc_roc = roc_auc_score(anomalies, scores)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_ranking.py", line 619, in roc_auc_score
    y_score = check_array(y_score, ensure_2d=False)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/l

Sub-LOF - AUC-ROC: 0.630, AUC-PR: 0.047

Running VALMOD for generated_data...
Error running VALMOD: Welcome to Matrix Profile

Attaching package: ‘tsmp’

The following object is masked from ‘package:base’:

    write

Reading data from /data/generated_data/dataset_7.test.csv
Error in if (motifs_per_size > 0) { : 
  missing value where TRUE/FALSE needed
Calls: valmod
Execution halted

Unexpected error running VALMOD for generated_data: Execution failed for VALMOD


...

Error in evaluation phase for STOMP: Input contains infinity or a value too large for dtype('float64').

Running Sub-LOF for generated_data...


Traceback (most recent call last):
  File "/var/folders/xh/bm_l2jz916s7t2p3pwrcr9y80000gn/T/ipykernel_25469/281630419.py", line 159, in <module>
    auc_roc, auc_pr = evaluate_algorithm(scores, test_file)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/xh/bm_l2jz916s7t2p3pwrcr9y80000gn/T/ipykernel_25469/2679600840.py", line 13, in evaluate_algorithm
    auc_roc = roc_auc_score(anomalies, scores)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_ranking.py", line 619, in roc_auc_score
    y_score = check_array(y_score, ensure_2d=False)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/l

Sub-LOF - AUC-ROC: 0.000, AUC-PR: 0.005

Running VALMOD for generated_data...
Error running VALMOD: Welcome to Matrix Profile

Attaching package: ‘tsmp’

The following object is masked from ‘package:base’:

    write

Reading data from /data/generated_data/dataset_15.test.csv
Error in if (motifs_per_size > 0) { : 
  missing value where TRUE/FALSE needed
Calls: valmod
Execution halted

Unexpected error running VALMOD for generated_data: Execution failed for VALMOD


Traceback (most recent call last):
  File "/var/folders/xh/bm_l2jz916s7t2p3pwrcr9y80000gn/T/ipykernel_25469/281630419.py", line 101, in <module>
    scores = run_algorithm(algo_name, test_file, custom_parameters=custom_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/xh/bm_l2jz916s7t2p3pwrcr9y80000gn/T/ipykernel_25469/1694792707.py", line 26, in run_algorithm
    raise RuntimeError(f"Execution failed for {algorithm_name}")
RuntimeError: Execution failed for VALMOD



Processing file: data/generated_data/dataset_5.test.csv in folder: generated_data

Running CBLOF for generated_data...
CBLOF - AUC-ROC: 0.620, AUC-PR: 0.035

Running COF for generated_data...
COF - AUC-ROC: 0.624, AUC-PR: 0.034

Running kMeans for generated_data...
kMeans - AUC-ROC: 0.722, AUC-PR: 0.040

Running KNN for generated_data...
KNN - AUC-ROC: 0.602, AUC-PR: 0.028

Running LOF for generated_data...
LOF - AUC-ROC: 0.564, AUC-PR: 0.027

Running PS-SVM for generated_data...
PS-SVM - AUC-ROC: 0.504, AUC-PR: 0.020

Running STAMP for generated_data...
STAMP - AUC-ROC: 0.881, AUC-PR: 0.177

Running STOMP for generated_data...
STOMP - AUC-ROC: 0.881, AUC-PR: 0.177

Running Sub-LOF for generated_data...
Sub-LOF - AUC-ROC: 0.765, AUC-PR: 0.056

Running VALMOD for generated_data...
VALMOD - AUC-ROC: 0.881, AUC-PR: 0.177

Processing file: data/generated_data/dataset_14.test.csv in folder: generated_data

Running CBLOF for generated_data...
CBLOF - AUC-ROC: 0.645, AUC-PR: 0.004

Running C